# Perceptual Loss For Real Time Transfer

该网络与原始版本的Style Transfer的套路是一样的，唯一的不同就是原始版本的style transfer是直接拿vgg-16，从一张噪音图开始，一方面比对content image，另一方面比对style image，不断调整噪音图，从而得到一张融合后的图。而Percept Loss For Real Time Transfer则是增加了一个auto encoder形状的网络，叫做transformation network，进原图出来预测图，这个预测图与原图在内容做比对，与风格图在风格上做比对。在训练的时候，有若干张内容图，只有一张风格图。这样就可以训练transformation network一边顾及到恢复原图，一边顾及到贴近特定的风格。  
结果就是，一个transformation network就贴近一张风格图。这样一来，一张新的内容图进来以后，直接就可以推断出风格迁移后的图像。图像质量理论上跟原始版本style transfer不会有啥差异，只不过是可以提前训练好该模型，不像style transfer原始版本，每次都要训练。

首先，拿一系列内容图+一张风格图训练一个网络，并保存下来

In [1]:
from main import build_parser
from train import network_train
from test import network_test
import os

# 当前目录
#current_directory = os.path.dirname(os.path.abspath(__file__))
current_directory = os.getcwd()


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 训练模式

parser = build_parser()
content_imagedir = os.path.join(current_directory, "../Images/original/")
style_imagepath = os.path.join(current_directory, "../Images/style-images/candy.jpg")
custom_args = [
    "--train-flag", "True",
    "--train-content", content_imagedir,
    "--train-style", style_imagepath,
    "--content-weight", "10",
    "--style-weight", "20",
    "--tv-weight", "3",
    "--max-iter", "15000"
]
args = parser.parse_args(custom_args)
print(args)

transform_network = network_train(args)

Namespace(batchs=8, check_iter=100, content_layers=[15], content_weight=10.0, cropsize=None, cuda_device_no=0, imsize=256, lr=0.1, max_iter=15000, model_load_path=None, output='stylized.jpg', save_path='./trained_models/', style_layers=[3, 8, 15, 22], style_weight=20.0, test_content=None, train_content='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/original/', train_flag=True, train_style='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/style-images/candy.jpg', tv_weight=3.0, vgg_flag='vgg16')
device:cuda:0


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Mon Nov 20 12:54:22 2023: iteration: [0/15000/],	content_loss: 0.23,	style_loss: 0.52,	tv_loss: 0.01,	total_loss: 12.65,	
Mon Nov 20 12:55:30 2023: iteration: [100/15000/],	content_loss: 11.00,	style_loss: 15.31,	tv_loss: 0.06,	total_loss: 416.41,	
Mon Nov 20 12:56:39 2023: iteration: [200/15000/],	content_loss: 11.62,	style_loss: 11.24,	tv_loss: 0.13,	total_loss: 341.44,	
Mon Nov 20 12:57:45 2023: iteration: [300/15000/],	content_loss: 10.64,	style_loss: 9.87,	tv_loss: 0.15,	total_loss: 304.31,	
Mon Nov 20 12:58:51 2023: iteration: [400/15000/],	content_loss: 10.40,	style_loss: 8.86,	tv_loss: 0.16,	total_loss: 281.68,	
Mon Nov 20 12:59:57 2023: iteration: [500/15000/],	content_loss: 10.63,	style_loss: 7.83,	tv_loss: 0.18,	total_loss: 263.46,	
Mon Nov 20 13:01:03 2023: iteration: [600/15000/],	content_loss: 10.89,	style_loss: 6.96,	tv_loss: 0.19,	total_loss: 248.65,	
Mon Nov 20 13:02:09 2023: iteration: [700/15000/],	content_loss: 11.04,	style_loss: 6.30,	tv_loss: 0.21,	total_loss: 237

现在开始测试

In [3]:
parser = build_parser()
model_path = os.path.join(current_directory, "trained_models/transform_network.pth")
test_imagepath = os.path.join(current_directory, "../Images/content-images/HeadBase.jpg")
output_imagepath = os.path.join(current_directory, "../Images/styletransfer-output/HeadBaseTransfered.jpg")
custom_args = [
    "--train-flag", "False",
    "--model-load-path", model_path,
    "--test-content", test_imagepath,
    "--output", output_imagepath
]

args = parser.parse_args(custom_args)
print(args)

network_test(args)


Namespace(batchs=8, check_iter=100, content_layers=[15], content_weight=1.0, cropsize=None, cuda_device_no=0, imsize=256, lr=0.1, max_iter=15000, model_load_path='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\trained_models/transform_network.pth', output='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/styletransfer-output/HeadBaseTransfered.jpg', save_path='./trained_models/', style_layers=[3, 8, 15, 22], style_weight=30.0, test_content='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/content-images/HeadBase.jpg', train_content=None, train_flag=False, train_style=None, tv_weight=1.0, vgg_flag='vgg16')
device:cuda:0
